In [4]:
import import_ipynb
from model import *

importing Jupyter notebook from model.ipynb
Loaded  400000  model
10
20


Using TensorFlow backend.


weights[0][1][3] = -0.3403
10
20
(20, 500, 60)
(20,)
(20, 60)
(20,)
(?, 500, 60, 50)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
sentence_indices (InputLayer)   (None, 60)           0                                            
__________________________________________________________________________________________________
Atriplets_indices (InputLayer)  (None, 500, 60)      0                                            
__________________________________________________________________________________________________
embedding (Embedding)           multiple             20000050    sentence_indices[0][0]           
                                                                 Atriplets_indices[0][0]          
__________________________________________________________________________________________________
AUnstackLayer (Lambda)  

In [6]:
def padding(max_len, x):
    x_len = x.shape[0]
    if(x_len >= max_len):
        return x[0:max_len]
    else:
        if(len(x.shape) > 1):
            dim = x.shape[1]
            a = x
            b = np.ones((max_len-x_len, dim))*-1
            return np.concatenate((a, b))
        else:
            a = x
            b = np.zeros(max_len-x_len)
            return np.concatenate((a, b))
        
def reshape_mask(max_len, x):
    dim=50
    padded_question_mask = padding(max_len, x)
    final_question_mask = []
    for qm in padded_question_mask:
        final_question_mask.append([qm]*dim)
    return final_question_mask
            
c = padding(500, data[4]["correct_triplets"])
print(c.shape)

NameError: name 'data' is not defined

In [7]:
dimOfQuestionVector = 300
dimOfTripletVector = 900

In [9]:
# create inputs for model
import glob
import os
import json
import re
import pickle

def clean(input):
    print(input)
    return ' '.join(re.sub(r'[^a-zA-Z]', ' ', input).split())
    #return re.sub(r'[^a-zA-Z]', ' ', input)

def clean_list(input):
    clean_output = []
    for triplet in input:
        clean_triplet = []
        for i in triplet:
            clean_triplet.append(clean(i))
        clean_output.append(np.asarray(clean_triplet))
    return clean_output

def prepare_data(data, question_max_words=60, triplets_max_numbers=500):
    question_vectors = []
    question_masks = []
    triplet_vectors = []
    Y = []
    for elem in data:
        clean_question = elem["question"]
        print(clean_question)
        clean_question_vector = padding(question_max_words, clean_question)
        clean_mask = elem["question_mask"]
        clean_mask_vector = reshape_mask(question_max_words, clean_mask)
        clean_correct_triplets = elem["correct_triplets"]
        clean_wrong_triplets = elem["wrong_triplets"]
        
        # for correct triplets
        correct_triplets_vectors = []
        for tr in clean_correct_triplets: # There are 500 triplets
            # Each tr has s, p and o. Each of the _s, _p and _o's are (num_tokens, 300) size
            _s = np.mean(tr[0], axis=0)
            _p = np.mean(tr[1], axis=0)
            _o = np.mean(tr[2], axis=0)
            correct_triplets_vectors.append(np.concatenate((_s, _p, _o))) # adding (900,) vector for each triplet
        triplet_vectors.append(padding(triplets_max_numbers, np.asarray(correct_triplets_vectors)))
        question_vectors.append(clean_question_vector)
        question_masks.append(clean_mask_vector)
        Y.append(1)

        # for wrong triplets
        wrong_triplets_vectors = []
        for tr in clean_wrong_triplets:
            _s = np.mean(tr[0], axis=0)
            _p = np.mean(tr[1], axis=0)
            _o = np.mean(tr[2], axis=0)
            wrong_triplets_vectors.append(np.concatenate((_s, _p, _o)))
        triplet_vectors.append(padding(triplets_max_numbers, np.asarray(wrong_triplets_vectors)))
        question_vectors.append(clean_question_vector)
        question_masks.append(clean_mask_vector)
        Y.append(0)
    return question_vectors, question_masks, triplet_vectors, Y

def generate_model_inputs():
    file_list = glob.glob('/Users/surthi/Downloads/train_2.ipwk')
    data = []
    for inputFile in file_list:
        f = open(inputFile, 'rb')
        test = pickle.load(f)
        data = np.append(data, test)
        f.close()
    
    return prepare_data(data, question_max_words=60, triplets_max_numbers=500)

question_vectors1, question_masks1, triplet_vectors1, Y1 = generate_model_inputs()#prepare_data(data, question_max_words=60, triplets_max_numbers=500)

[[-4.4058e-02  3.6611e-01  1.8032e-01 ...  1.8625e-01 -9.7817e-02
  -6.7104e-05]
 [ 2.7332e-01 -4.2871e-03  3.2323e-02 ... -2.7807e-02  4.0019e-02
   2.3103e-01]
 [ 8.9187e-02  2.5792e-01  2.6282e-01 ...  1.4421e-01 -1.6900e-01
   2.6501e-01]
 ...
 [-5.9446e-02 -6.9223e-02 -9.7491e-02 ... -6.8324e-02  1.0945e-01
   1.7945e-01]
 [ 1.2001e-02  2.0751e-01 -1.2578e-01 ...  1.3871e-01 -3.6049e-01
  -3.5000e-02]
 [-5.9922e-01 -1.4993e-01 -2.6709e-01 ... -3.6858e-02 -1.7136e-02
  -3.8951e-02]]
[[ 0.27204001 -0.06203    -0.1884     ...  0.13015001 -0.18317001
   0.1323    ]
 [ 0.31970999 -0.18574999  0.13301    ...  0.13639    -0.44457999
   0.45965999]
 [-0.0066796   0.22238     0.27709001 ...  0.059372    0.0014085
   0.098727  ]
 ...
 [ 0.19410001  0.22603001 -0.43764001 ...  0.091957    0.38631999
   0.11736   ]
 [-0.082752    0.67203999 -0.14986999 ... -0.1918     -0.37845999
  -0.06589   ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]
[[ 0.0628   -0.043592 -0.

[[-4.40579988e-02  3.66109997e-01  1.80319995e-01 ...  1.86250001e-01
  -9.78169963e-02 -6.71040034e-05]
 [-1.47119999e-01  2.07580000e-01 -2.15890005e-01 ...  1.07380003e-01
   7.07430020e-02  4.33299989e-01]
 [-9.95339975e-02  2.82019991e-02 -2.31889993e-01 ... -1.03019997e-01
  -3.99640016e-02  2.68079996e-01]
 ...
 [ 2.46539995e-01  1.03390001e-01 -4.12090003e-01 ...  8.21510032e-02
   1.33320004e-01  2.03700006e-01]
 [-4.40579988e-02  3.66109997e-01  1.80319995e-01 ...  1.86250001e-01
  -9.78169963e-02 -6.71040034e-05]
 [-1.47119999e-01  2.07580000e-01 -2.15890005e-01 ...  1.07380003e-01
   7.07430020e-02  4.33299989e-01]]
[[-0.084961   0.502      0.0023823 ... -0.21511   -0.26304   -0.0060173]
 [ 0.066466  -0.0084298 -0.57305   ... -0.19774    0.045826  -0.18881  ]
 [ 0.31924    0.06316   -0.27858   ...  0.082745   0.097801   0.25045  ]
 ...
 [-0.069323  -0.21445   -0.20355   ...  0.066121   0.07302    0.040155 ]
 [ 0.24236   -0.054326  -0.12821   ... -0.5052     0.25838   -0.685

[[-0.38126001 -0.094614    0.92057002 ... -0.50537997  0.33085999
   0.016369  ]
 [-0.082752    0.67203999 -0.14986999 ... -0.1918     -0.37845999
  -0.06589   ]
 [ 0.089187    0.25792     0.26282001 ...  0.14421    -0.169
   0.26501   ]
 ...
 [-0.082752    0.67203999 -0.14986999 ... -0.1918     -0.37845999
  -0.06589   ]
 [-0.37814999  0.18757001  0.34807    ... -0.016685   -0.36741999
  -0.23689   ]
 [ 0.060216    0.21799    -0.04249    ...  0.11709    -0.16692001
  -0.094085  ]]
[[ 0.17772999 -0.27403    -0.4332     ...  1.05680001  0.81264001
  -0.52161002]
 [-0.18567     0.066008   -0.25209001 ... -0.023452    0.12302
   0.3312    ]
 [ 0.62467998 -0.10042    -0.10904    ...  0.42717001 -0.029018
  -0.0067072 ]
 ...
 [ 0.060216    0.21799    -0.04249    ...  0.11709    -0.16692001
  -0.094085  ]
 [ 0.27204001 -0.06203    -0.1884     ...  0.13015001 -0.18317001
   0.1323    ]
 [ 0.003604    0.052964   -0.22904    ...  0.16590001  0.47940999
  -0.17137   ]]
[[ 0.01332  -0.051085 -0.1

[[ 0.51668    -0.30767     0.49629    ... -0.12564     0.36098
  -0.19307   ]
 [ 0.089187    0.25792     0.26282    ...  0.14421    -0.169
   0.26501   ]
 [-0.29638    -0.54261     0.17038    ...  0.17604     0.20929
  -0.1377    ]
 ...
 [ 0.012001    0.20751    -0.12578    ...  0.13871    -0.36049
  -0.035     ]
 [-0.20996    -0.15577    -0.37736    ... -0.55673     0.00072776
   0.065055  ]
 [-0.045684    0.11925    -0.56572    ...  0.52247     0.045819
   0.071901  ]]
[[ 0.20934001 -0.78591001 -0.048814   ...  0.31116    -0.23563001
   0.16366   ]
 [ 0.39155999  0.12067     0.022005   ...  0.34562999 -0.17935
  -0.14093   ]
 [ 0.04965     0.052936   -0.19768    ...  0.09219     0.11284
  -0.020346  ]
 ...
 [ 0.012001    0.20750999 -0.12578    ...  0.13871001 -0.36048999
  -0.035     ]
 [-0.42462     0.029553    0.025563   ... -0.10931     0.69296002
  -0.15617   ]
 [-0.20638999 -0.11403    -0.11346    ... -0.20819999  0.30700001
  -0.25051999]]
[[-0.72635    0.59717   -0.25093   ...

[[ 0.23101     0.16445    -0.7665     ... -0.00082625 -0.35804
   0.11133   ]
 [-0.084961    0.502       0.0023823  ... -0.21511    -0.26304
  -0.0060173 ]
 [ 0.27204    -0.06203    -0.1884     ...  0.13015    -0.18317
   0.1323    ]
 ...
 [-0.18024     0.0084106  -0.30463    ... -0.38552     0.24324
  -0.17578   ]
 [-0.21665     0.39423    -0.35427    ... -0.31114     0.18653
   0.13374   ]
 [ 0.012001    0.20751    -0.12578    ...  0.13871    -0.36049
  -0.035     ]]
[[ 0.067993    0.012205    0.58143002 ... -0.1532     -0.017811
   0.17878   ]
 [-0.22121    -0.053349   -0.49312001 ... -0.14574    -0.46882001
   0.089175  ]
 [ 0.012001    0.20750999 -0.12578    ...  0.13871001 -0.36048999
  -0.035     ]
 ...
 [ 0.012001    0.20750999 -0.12578    ...  0.13871001 -0.36048999
  -0.035     ]
 [ 0.1177     -0.17744     0.52218002 ...  0.48631001 -0.090661
  -0.31046   ]
 [ 0.012001    0.20750999 -0.12578    ...  0.13871001 -0.36048999
  -0.035     ]]
[[ 0.31924    0.06316   -0.27858   ...

In [10]:
print("Hi")

Hi


In [313]:
questions_input_shape = (maxWordsPerSentence, dimOfQuestionVector, )
questions_mask_input_shape = (maxWordsPerSentence, 50, )
triplets_input_shape = (maxTriplets, dimOfTripletVector, )

# question_vectors(, 60, 300), question_masks(, 60, 300), triplet_vectors (,500,900), Y
len(question_vectors[0][0])

300

In [292]:
# question inputs
question_vectors_input = Input(shape=questions_input_shape, dtype='float32', name="question_indices_input")   
question_masks_input = Input(shape=questions_mask_input_shape, dtype='float32', name="question_masks_input")

# triplets input
triplets_input = Input(shape=triplets_input_shape, dtype='float32', name="triplets_input")  #(None, 500, 900)

# layers
# gru1 = Bidirectional(GRU( 50, return_sequences=True, name="Sentence_GRU1"))
# gru2 = Bidirectional(GRU( 50, return_sequences=False, name="Sentence_GRU2"))
# tgru1 = Bidirectional(GRU( 50, return_sequences=True, name="Triplet_GRU1"))
# tgru2 = Bidirectional(GRU( 50, return_sequences=False, name="Triplet_GRU2"))

gru1 = (GRU( 50, return_sequences=True, name="Sentence_GRU1"))
gru2 = (GRU( 50, return_sequences=True, name="Sentence_GRU2"))
tgru1 = (GRU( 50, return_sequences=True, name="Triplet_GRU1"))
tgru2 = (GRU( 50, return_sequences=False, name="Triplet_GRU2"))
mean_layer = Lambda(lambda xin: mean(xin, axis=1), name="AMeanLayer")

X_question = gru2(gru1(question_vectors_input))
X_question = Multiply()([question_masks_input, X_question]) # (None, 60, 50)
X_question = mean_layer(X_question)

X_triplets = Dense(50, activation='relu', name="Dense50T")(triplets_input)#tgru2(tgru1(triplets_input))
X_triplets = tgru2(tgru1(X_triplets))
X_concatenated = Concatenate()([X_question, X_triplets])
X_concatenated = Dense(64, activation='relu', name="Dense64")(X_concatenated)
Y_pred = Dense(1, activation='sigmoid', name="Dense1")(X_concatenated)

m.v3 = Model(inputs=[question_vectors_input, question_masks_input, triplets_input], outputs=Y_pred)
print(m.v3.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
question_indices_input (InputLa (None, 60, 300)      0                                            
__________________________________________________________________________________________________
Sentence_GRU1 (GRU)             (None, 60, 50)       52650       question_indices_input[0][0]     
__________________________________________________________________________________________________
triplets_input (InputLayer)     (None, 500, 900)     0                                            
__________________________________________________________________________________________________
question_masks_input (InputLaye (None, 60, 50)       0                                            
__________________________________________________________________________________________________
Sentence_G

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(inputs=[question_vectors, question_masks, triplet_vectors], Y)

In [70]:
# from keras.layers import Dense, Input, Dropout, LSTM, Activation, SimpleRNN, GRU, Concatenate, Multiply, Reshape, Flatten, Bidirectional

# # layers
# s1_bilstm = Bidirectional(GRU(128, return_sequences = True), name = "s1_bilstm")
# s2_bilstm = Bidirectional(GRU(128, return_sequences = False), name = "s2_bilstm")
# t_bilstm = Bidirectional(GRU(128, return_sequences = False), name = "t_bilstm")
# embedding_layer = pretrained_embedding_layer(word_to_vec_map, words_to_index)
# unstack_layer = Lambda(lambda xin: myUnstack(xin), name="AUnstackLayer")

# # sentence inputs
# sentence_indices = Input(shape=(maxWordsPerSentence,), dtype='int32', name="sentence_indices")   
# sentence_masks = Input(shape=(2*lstm_dim,), dtype='float32', name="sentence_masks")

# # sentence embeddings
# sentence_embeddings = embedding_layer(sentence_indices)       
# # X_sentence = Reshape((maxWordsPerSentence, -1), name="s_reshape")(sentence_embeddings)
# X_sentence = s1_bilstm(sentence_embeddings)
# X_sentence = Multiply()([sentence_masks, X_sentence]) # (None, 60, 256)
# X_sentence = s2_bilstm(X_sentence)

# # triplets
# triplets_input = Input(shape=(maxTriplets, maxWordsPerSentence,), dtype='int32', name="tripletz") 
# triplet_embeddings = embedding_layer(triplets_input)

# X_triplets = Reshape((maxTriplets, -1))(triplet_embeddings)
# X_triplets = t_bilstm(X_triplets)

# X_concatenated = Concatenate()([X_sentence, X_triplets])
# X_concatenated = Dense(256, activation='relu', name="Dense256")(X_concatenated)
# X_concatenated = Dense(64, activation='relu', name="Dense64")(X_concatenated)
# Y_pred = Dense(1, activation='sigmoid', name="Dense1")(X_concatenated)

# m.v2 = Model(inputs=[sentence_indices, sentence_masks, triplets_input], outputs=X_concatenated)
# print(m.v2.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
sentence_indices (InputLayer)   (None, 60)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           multiple             20000050    sentence_indices[0][0]           
                                                                 tripletz[0][0]                   
__________________________________________________________________________________________________
tripletz (InputLayer)           (None, 500, 60)      0                                            
__________________________________________________________________________________________________
sentence_masks (InputLayer)     (None, 256)          0                                            
__________